In [1]:
import wikipedia
import warnings
import spacy
import geonamescache
import csv
import pandas as pd

In [2]:
# get the city list

gc = geonamescache.GeonamesCache()

cities = gc.get_cities()

def gen_dict_extract(var, key):
    if isinstance(var, dict):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, (dict, list)):
                yield from gen_dict_extract(v, key)
    elif isinstance(var, list):
        for d in var:
            yield from gen_dict_extract(d, key)

cities = [*gen_dict_extract(cities, 'name')]

In [3]:
def get_city(string): 
    #(string) -> string
    #once we input a string, it will return a city most relate to this string. 
    #If we can't find a city relate to this string, it will return nothing.
    #For example : get_city('Peter Parker') -> 'New York City'
  warnings.filterwarnings('ignore')
  nlp = spacy.load('en')

  try:
    # get page summary from wikipedia
    page = wikipedia.page(string)
    tem =str(page.summary)
    doc = nlp(tem)
    sentences = list(doc.sents)
    ls = []
    # get the location words
    for sentence in sentences:
      process = nlp(sentence.text).ents
      for token in process:
        if token.text in cities:
          ls.append(token.text)
    if len(ls) != 0:
        frequent_location = max(ls,key=ls.count) 
        return(frequent_location)
  #check whether the related page exist
  except wikipedia.exceptions.PageError:
    pass
  # to deal with vague input 
  except wikipedia.exceptions.DisambiguationError as e:
    get_city(e.options[0])

In [4]:
get_city('Peter Parker')

'New York City'

In [5]:
word_ls = []
city_ls = []

In [6]:
#get the trending words of Twitter
file = open("./trending_word.txt")
#save the trending words and its related city to list
lines = file.readlines()
for line in lines:
    #print(line)
    tem = get_city(line.strip())
    if tem != None:
        word_ls.append(line)
        city_ls.append(tem)

In [9]:
#save the trending words and its related city to csv file
dataframe = pd.DataFrame({'trending word':word_ls,'related city':city_ls})

dataframe.to_csv("demo.csv",index=False)